In [3]:
import json
from typing import Dict, Any
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
import requests
from pydantic import BaseModel, Field
import os
import httpx


async def get_properties():
    url = 'http://localhost:9000/collections/public.5001elveg2.0l'
    try:
        async with httpx.AsyncClient() as client:
            res = await client.get(url)
            res.raise_for_status()
            geojson_response = res.json()
            properties = 
            for feature in geojson_response.get('features', []):
                properties.update(feature.get('properties', {}).keys())

            return {'properties': list(properties)}
    except httpx.HTTPStatusError as e:
        return f"Error response {e.response.status_code} while requesting {e.request.url!r}."
    
await get_properties()

{'properties': []}

In [4]:
class GetOGCAPIFeaturesPropertiesInput(BaseModel):
    """Input for GetOGCAPIFeaturesPropertiesTool."""
    collection_url: str = Field(
        ..., description="The URL for the collection items, i.e., http://{BASE_URL}/collections/{collection_id}")
    cql: str = Field(..., description=("CQL (common query language) to be used for filtering like:\n"
                                       "'http://{BASE_URL}/collections/{collection_id}/items?filter=< cql goes here >"))


class GetOGCAPIFeaturesPropertiesTool(BaseTool):
    """Tool"""

    name: str = "ogc_api_features_collection_properties"
    args_schema: Type[BaseModel] = GetOGCAPIFeaturesPropertiesInput
    description: str = (
        "Use this tool to retrieve all properties of features from an OGC API Features collection.\n"
        "Use CQL (Common Query Language) to filter the returned features.\n"
        "The properties will be returned as a list."
    )

    def _run(self, collection_url: str, cql: str, *args: Any, **kwargs: Any) -> Any:
        url = f'{collection_url}/items?filter={cql}'
        try:
            geojson_response = requests.get(url).json()
            properties = set()
            for feature in geojson_response.get('features', []):
                properties.update(feature.get('properties', {}).keys())

            return {'properties': list(properties)}
        except requests.RequestException as e:
            return {'error': str(e)}

    async def _arun(self, collection_url: str, cql: str, *args: Any, **kwargs: Any) -> Any:
        url = f'{collection_url}/items?filter={cql}&limit=5000'
        try:
            async with httpx.AsyncClient() as client:
                res = await client.get(url)
                res.raise_for_status()
                geojson_response = res.json()
                properties = set()
                for feature in geojson_response.get('features', []):
                    properties.update(feature.get('properties', {}).keys())

                return {'properties': list(properties)}
        except httpx.HTTPStatusError as e:
            return f"Error response {e.response.status_code} while requesting {e.request.url!r}."

In [39]:
GetOGCAPIFeaturesPropertiesTool

set()

In [41]:
from typing import Sequence

agent_tools = [GetOGCAPIFeaturesPropertiesTool()]

uninvoked_tools = [GetOGCAPIFeaturesPropertiesTool]

def get_tool_names(tools: list[BaseTool]):
    overlapping_tools = filter(lambda t: type(t).__name__ in [
                           uit.__name__ for uit in uninvoked_tools], agent_tools)
    return list(map(lambda t: t.name, overlapping_tools))


get_tool_names(uninvoked_tools)

['ogc_api_features_collection_properties']